In [1]:
import os
from random import randint
import pandas as pd
import seaborn as sns

In [2]:
folder = os.path.join(os.getcwd(),'..','data','korpus')
vanilla = os.path.join(folder,'ngram','vanilla')
unk = os.path.join(folder,'ngram','unk')

In [ ]:
df_test

In [5]:
def generate_text(phrase:str, model:str, n:int):
    
    if model == 'vanilla' or model == 'laplace':
        model_path = vanilla
    elif model == 'unk': 
        model_path = unk
    else: raise Exception('Model does not exist!')
    
    print("Loading Models: ",end='')
    if n in [1,2,3]:
        xgrams = ['unigram.csv','bigram.csv','trigram.csv']
        ngram_path = xgrams[n-1]
        
        xgrams_types = ['Unigram','Bigram','Trigram']
        ngram_type = xgrams_types[n-1]
        
        df_ngram = pd.read_csv(os.path.join(model_path, ngram_path))
        
        ngrams = df_ngram[ngram_type].unique()
        
        if n != 1:
            prev_df_gram = pd.read_csv(os.path.join(model_path,xgrams[n-2]))
            prev_ngrams = prev_df_gram[xgrams_types[n-2]].unique()
            
    else: raise Exception('Choose Unigram, Bigram or Trigram!')
    print('[OK]')
    print('Generating Sentence...')
    
    generated_word = ""
    
    if n == 1:
        top_words = df_ngram['Probability'].astype(float).nlargest(50).index
    
    while generated_word != '</s>':
        phrase += ' '
        if n == 1:
            generated_word = df_ngram.iat[top_words[randint(0,9)],0]
            phrase += generated_word
        
        else:
            #Get previous words
            tokens = phrase.split(' ')
            history = ';'.join(tokens[len(tokens)-n:len(tokens)-1])
            
            #Find most probable words that follow previous words          
            if model == 'vanilla' or history in prev_ngrams:
                top_words = df_ngram[df_ngram[ngram_type].str.startswith(history)]['Probability'].astype(float).nlargest(10).index
            
            #If there is no match for the history, then laplace smoothing comes in.
            #Laplace smoothing will ensure that there is an occurence with the previous history at least once 
            #with every other word. Hence the next word becomes essentially a random guess.
            else: 
                top_words = [randint(0,len(df_ngram)-1)]
            
            #Pick a random word from the top 10.
            generated_word = df_ngram.iat[top_words[randint(0,len(top_words)-1)],0].split(';')[-1]
            print(generated_word, end=' ')
            #Add to current phrase
            phrase += generated_word
            
    return phrase


In [6]:
%%time
generate_text('Kulur ikrah', 'unk', 2)

Loading Models: [OK]
Generating Sentence...


KeyError: 'Probability'